In [17]:
from dotenv import load_dotenv
import os
import pandas as pd
import alpaca_trade_api as tradeapi
import requests
import json
import time
load_dotenv("alphavantage_api_key.env")

True

In [32]:
symbol_list = ["ALK", "UAL", "LUV", "DAL", "AL"]
tech_list = ["AAPL", "GOOGL", "AMZN", "MSFT", "FB"]

for symbol in symbol_list:
    content = requests.get(url= "https://www.alphavantage.co/query?",  params = {"function":"BALANCE_SHEET", "symbol": symbol, "apikey": key})
    data = content.json()
    data_test=data['quarterlyReports']
    df = pd.DataFrame.from_dict(data_test)
    df.set_index('fiscalDateEnding', inplace=True)
    
    file_name = f'Data/Airlines/{symbol}_quarterly_reports.csv'
    df.to_csv(file_name)


In [33]:
symbol_list = ["AAL", "UAL", "LUV", "DAL", "AL"]
tech_list = ["AAPL", "GOOGL", "AMZN", "MSFT", "FB"]

for symbol in symbol_list:
    stock_price = requests.get('https://www.alphavantage.co/query?', params={'function':'TIME_SERIES_MONTHLY_ADJUSTED','symbol':symbol,
                                                                              'apikey':'Alpha_vantage_Api_key'})
    stock_price = stock_price.json()
    try:
        stock_price = stock_price['Monthly Adjusted Time Series']
    except KeyError:
        print('KeyError. Trying again in 60 seconds...')  
        time.sleep(60)
        stock_price = requests.get('https://www.alphavantage.co/query?', params={'function':'TIME_SERIES_MONTHLY_ADJUSTED','symbol':symbol,
                                                                              'apikey':'Alpha_vantage_Api_key'})
        stock_price = stock_price['Monthly Adjusted Time Series']

    data = {}
    date_list = []
    close_list = []
    for date in stock_price:
        date_list.append(date)
        close_list.append(stock_price[date]['4. close'])
    
    data = {'date': date_list, 'close': close_list}
    price_df = pd.DataFrame.from_dict(data)
    
    price_df.set_index('date', inplace=True)
    file_name = f'Data/Prices/{symbol}.csv'
    price_df.to_csv(file_name)

In [31]:
symbol_list = ["AAPL"]
#symbol_list = ["AAL", "UAL", "LUV", "DAL", "AL"]
for symbol in symbol_list:
    stock_price = requests.get('https://www.alphavantage.co/query?', params={'function':'TIME_SERIES_MONTHLY_ADJUSTED','symbol':symbol,
                                                                              'apikey':'Alpha_vantage_Api_key'})
    stock_price = stock_price.json()
    try:
        stock_price = stock_price['Monthly Adjusted Time Series']
    except KeyError:
        print('KeyError. Trying again in 60 seconds...')  
        time.sleep(60)
        stock_price = requests.get('https://www.alphavantage.co/query?', params={'function':'TIME_SERIES_MONTHLY_ADJUSTED','symbol':symbol,
                                                                              'apikey':'Alpha_vantage_Api_key'})
        stock_price = stock_price['Monthly Adjusted Time Series']
    data = {}
    date_list = []
    close_list = []
    for date in stock_price:
        data[date] = stock_price[date]['4. close']
        date_list.append(date)
        close_list.append(stock_price[date]['4. close'])
    data = {'date': date_list, 'close': close_list}
    price_df = pd.DataFrame.from_dict(data)
    price_df.columns = ['date', 'close']
    price_df.set_index('date', inplace=True)
    price_df_ascending = price_df.sort_values(by="date", ascending=True)
    price_df_average = price_df_ascending.rolling(3).mean() 
    price_df_average = price_df_ascending.iloc[::3, :].truncate(before = "2019-03-29")
    final_month = price_df_ascending.iloc[-1] # we wanna append final_month to price_df_average
    price_df_average.append(final_month) 
    file_name = f'Data/Prices/{symbol}.csv'
    price_df.to_csv(file_name)
    print(f'{symbol}:')
    print(price_df_average)
    print('---')
    print(final_month)

1
AAPL:
               close
date                
2019-03-29  189.9500
2019-06-28  197.9200
2019-09-30  223.9700
2019-12-31  293.6500
2020-03-31  254.2900
2020-06-30  364.8000
2020-09-30  115.8100
---
close    108.8600
Name: 2020-10-30, dtype: object
